In [1]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client Django


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 54.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
from django.http import HttpResponse
from django.views import View
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials


In [4]:
CLIENT_SECRETS_FILE = '/content/client_secret.com (1).json'


In [7]:
class GoogleCalendarInitView(View):
    def get(self, request):
        flow = Flow.from_client_secrets_file(
            CLIENT_SECRETS_FILE,
            scopes=['https://www.googleapis.com/auth/calendar.readonly'],
            redirect_uri='urn:ietf:wg:oauth:2.0:oob',
        )
        authorization_url, state = flow.authorization_url(access_type='offline', include_granted_scopes='true')
        request.session['state'] = state
        return HttpResponse(authorization_url)

class GoogleCalendarRedirectView(View):
    def get(self, request):
        state = request.session['state']
        flow = Flow.from_client_secrets_file(
            CLIENT_SECRETS_FILE,
            scopes=['https://www.googleapis.com/auth/calendar.readonly'],
            redirect_uri='urn:ietf:wg:oauth:2.0:oob',
            state=state
        )
        flow.fetch_token(authorization_response=request.GET.get('code'))
        credentials = flow.credentials

        service = build('calendar', 'v3', credentials=credentials)
        events_result = service.events().list(calendarId='primary', maxResults=10).execute()
        events = events_result.get('items', [])

        response_data = []
        for event in events:
            response_data.append({
                'summary': event['summary'],
                'start': event['start'].get('dateTime', event['start'].get('date')),
                'end': event['end'].get('dateTime', event['end'].get('date'))
            })
        return HttpResponse(response_data)


In [6]:
!pip install django
!pip install djangorestframework

import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")

!python manage.py migrate

!python manage.py runserver


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
python3: can't open file '/content/manage.py': [Errno 2] No such file or directory
python3: can't open file '/content/manage.py': [Errno 2] No such file or directory
